# Project 4: Advanced Lane Finding

## Overview
In this project we would improve our technique for finding road lanes and curvature of the lanes. In order to do this, we would use a series of steps, outlined below:

1. Camera Calibration
2. Distortion Correction
3. Color and Gradient Thresholds
4. Perspective Transform
5. Detect Lane Lines
6. Determine Lane Curvature
7. Apply these techniques to the frames of the video

In this particular notebook, we would approach number 7., applying all the building blocks to the video frames!

## 7. Apply these techniques to the frames of the video

Let's start by moving over all the methods defined in the development notebook, getting them ready to use in the pipeline.

In [1]:
# Imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import os

%matplotlib inline

In [2]:
calibration_params_file = 'dist_pickle.p'

with open(calibration_params_file, 'rb') as f:
    calibration_params = pickle.load(f)

mtx = calibration_params["mtx"]
dist = calibration_params["dist"]

In [3]:
def undistort(img):
    return cv2.undistort(img, mtx, dist, None, mtx)

In [4]:
test_img = mpimg.imread('test_images/straight_lines1.jpg')
test_img_size = (test_img.shape[1], test_img.shape[0])

x1, y1 = (500, 480)
x2, y2 = (780, 480)
x3, y3 = (1200, 670)
x4, y4 = (80, 670)

src = np.float32(
    [[x1, y1],
     [x2, y2],
     [x3, y3],
     [x4, y4]])

offset = 10
dst = np.float32(
    [[offset, offset],
     [test_img_size[0] - offset, offset],
     [test_img_size[0] - offset, test_img_size[1] - offset],
     [offset, test_img_size[1] - offset]])

M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst, src)

In [5]:
def warp(img):
    img_size = (img.shape[1], img.shape[0])
    
    img = undistort(img)
    warped_img = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped_img

def unwarp(warped, img_size):
    return cv2.warpPerspective(warped, M_inv, img_size, flags=cv2.INTER_LINEAR)

In [6]:
def line_color_filter(warped_img):
    hsv = cv2.cvtColor(warped_img, cv2.COLOR_RGB2HSV).astype(np.float)
    
    yellow_mask = cv2.inRange(hsv, np.array([20, 50, 50]), np.array([40, 255, 255]))
    white_mask = cv2.inRange(hsv, np.array([0, 0, 200]), np.array([255, 70, 255]))
    
    return cv2.bitwise_and(warped_img, warped_img, mask= white_mask | yellow_mask)

In [7]:
def abs_sobel_thresh(img_channel, orient='x', sobel_kernel=3, thresh=(0, 255)):
    sobel = np.zeros(img_channel.shape)
    
    if orient == 'x':
        sobel = cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient == 'y':
        sobel = cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary_output

def mag_sobel_thresh(img_channel, sobel_kernel=3, thresh=(0, 255)):
    sobelx = cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    magn_sobel = np.sqrt(sobelx ** 2 + sobely ** 2)
    scaled_sobel = np.uint8(255 * magn_sobel / np.max(magn_sobel))
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary_output

def dir_sobel_thresh(img_channel, sobel_kernel=3, thresh=(0, np.pi/2)):
    sobelx = cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    gradient_direction = np.uint8(np.arctan2(abs_sobely, abs_sobelx))
    
    binary_output = np.zeros_like(gradient_direction)
    binary_output[(gradient_direction >= thresh[0]) & (gradient_direction <= thresh[1])] = 1

    return binary_output

In [8]:
# Process image to detect the lines by applying a mixture of gradient thresholds.
def lines_preprocessing(img, s_thresh=(200, 255), sx_thresh=(50, 100)):
    copy = np.copy(img)
    
    # Convert image to both HSV and HLS
    hsv = cv2.cvtColor(copy, cv2.COLOR_RGB2HSV).astype(np.float)
    hls = cv2.cvtColor(copy, cv2.COLOR_RGB2HLS).astype(np.float)
       
    # Grab the HLS Saturation channel
    s_channel = hls[:,:,2]
    # Grab the HSV Value channel
    v_channel = hsv[:,:,2]
    
    gradx = abs_sobel_thresh(v_channel, orient='x', thresh=sx_thresh)
    grady = abs_sobel_thresh(v_channel, orient='y', thresh=sx_thresh)
    mag_binary = mag_sobel_thresh(v_channel, sobel_kernel=5, thresh=(200, 255))
    dir_binary = dir_sobel_thresh(v_channel, sobel_kernel=5, thresh=(np.pi/4, 3 * np.pi / 4))
    
    combined_thresh = np.zeros_like(v_channel)
    combined_thresh[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    # Threshold Saturation channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    color_binary = np.dstack((np.zeros_like(combined_thresh), combined_thresh, s_binary))
    return color_binary

In [9]:
def convert_to_binary(channels_binary_img):
    binary_img = np.zeros_like(channels_binary_img[:,:,1])
    binary_img[(channels_binary_img[:,:,1] == 1) | (channels_binary_img[:,:,2] == 1)] = 1
    return binary_img

In [10]:
def get_histogram(binary_img, factor=1.5):
    return np.sum(binary_img[int(binary_img.shape[0]//factor):,:], axis=0)

In [11]:
def get_line_fits(lines_img, left_line, right_line):
    # Convert warped masked image to binary
    lines_img = np.uint8(lines_img)
    binary_img = convert_to_binary(lines_img)

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = get_histogram(binary_img)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_img, binary_img, binary_img)) * 255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_img.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_img.shape[0] - (window+1)*window_height
        win_y_high = binary_img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # If we did not find pixels for one of the lines (left/right), return false, i.e. we couldn't find the line
    if lefty.size == 0 or leftx.size == 0 or righty.size == 0 or rightx.size == 0:
        return False
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    left_line.fit = left_fit
    right_line.fit = right_fit

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_img.shape[0]-1, binary_img.shape[0] )
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    
    left_line.ploty = ploty
    right_line.ploty = ploty
    
    left_line.recent_fitx.append(left_fitx)
    right_line.recent_fitx.append(right_fitx)
    left_line.average_fits()
    right_line.average_fits()
    
    left_line.fitx = left_fitx
    right_line.fitx = right_fitx
    
    return True

In [12]:
def get_next_line_fits(lines_img, left_line, right_line):
    # Convert warped masked image to binary
    lines_img = np.uint8(lines_img)
    binary_img = convert_to_binary(lines_img)
    
    nonzero = binary_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    
    left_fit = left_line.fit
    right_fit = right_line.fit
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin))) 
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # If we did not find pixels for one of the lines (left/right), return false, i.e. we couldn't find the line
    if lefty.size == 0 or leftx.size == 0 or righty.size == 0 or rightx.size == 0:
        return False
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    left_line.fit = left_fit
    right_line.fit = right_fit
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_img.shape[0] - 1, binary_img.shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    
    left_line.ploty = ploty
    right_line.ploty = ploty
    
    left_line.recent_fitx.append(left_fitx)
    right_line.recent_fitx.append(right_fitx)
    left_line.average_fits()
    right_line.average_fits()
    
    left_line.fitx = left_fitx
    right_line.fitx = right_fitx
    
    return True

In [13]:
def draw_lines(img, lines_img, left_line, right_line):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(lines_img[:,:,0]).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_line.avg_fitx, left_line.ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.avg_fitx, right_line.ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix
    newwarp = unwarp(color_warp, img_size=(img.shape[1], img.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    return result

In [14]:
def get_curvature(left_line, right_line):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30 / 720 # meters per pixel in y dimension
    xm_per_pix = 3.7 / 700 # meters per pixel in x dimension
    
    ploty = left_line.ploty
    y_eval = np.max(ploty)
    
    # Fit new polynomials to x, y in world space
    left_fit_cr = np.polyfit(ploty * ym_per_pix, left_line.avg_fitx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, right_line.avg_fitx * xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * right_fit_cr[0])
    
    left_line.curvature = left_curverad
    right_line.curvature = right_curverad

In [15]:
def get_center_offset(img, left_line, right_line):
    # Get the center area in between the 2 lines (i.e. the lane area)
    lane_center = left_line.fitx[-1] + (right_line.fitx[-1] - left_line.fitx[-1]) / 2 # we look at the last most pixels from each line
    xm_per_pix = 3.7 / img.shape[1]
    # Car's location is at the center of the image's x axis since the camera is in the center of the front screen
    car_center = img.shape[1] / 2
    
    # Now calculate the offset and scale to meters
    offset = (lane_center - car_center) * xm_per_pix
    
    # Negative offset means the car is to the right of the lane center
    # Positive offset means the car is to the left of the lane center
    left_line.center_offset = offset
    right_line.center_offset = offset

In [16]:
font_face = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 3
def write_measurements(result_img, left_line, right_line):
    avg_curvature = (left_line.curvature + right_line.curvature) / 2
    
    curvature_text = 'Curvature radius of the lane: ' + str(avg_curvature) + ' m.'
    offset_text = 'Vehicle offset from center of lane: ' + str(left_line.center_offset) + ' m.'
    
    cv2.putText(result_img, curvature_text, (50, 50), font_face, font_scale, (163, 0, 0), thickness)
    cv2.putText(result_img, offset_text, (50, 80), font_face, font_scale, (163, 0, 0), thickness)

    return result_img

Ok, now that we have all the individual building blocks, let's start to write the video processing units.

First, let's create a helper class that will keep track of all the important params.

In [28]:
from collections import deque

class Line():
    MAX_ITERATIONS = 5
    MAX_RETRIES = 0
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False
        # Number of frames after which we restart  the search from scratch
        self.retries = self.MAX_RETRIES
        # Flag to indicate if this is the first frame
        self.is_first_frame = True
        
        # x values of the last n fits of the line
        self.recent_fitx = deque(maxlen=self.MAX_ITERATIONS)
        # average x values of the fitted line over the last n iterations
        self.avg_fitx = None     

        # current y values
        self.ploty = None
        # current x values
        self.fitx = None
        # current fit
        self.fit = None

        # radius of curvature of the line in some units
        self.curvature = None 
        # distance in meters of vehicle center from the line
        self.center_offset = None
    
    def average_fits(self):
        self.avg_fitx = np.mean(self.recent_fitx, axis=0)
    
    def reset_retries(self):
        self.retries = self.MAX_RETRIES
    
    def has_previous_fit(self):
        return self.avg_fitx is not None

In [24]:
global left_line
global right_line
left_line = None
right_line = None
def process_image(image):
    global left_line
    global right_line
    # Initialize the line objects in case this is the first frame
    if left_line is None:
        left_line = Line()
    if right_line is None:
        right_line = Line()
    
    # Process the image: warping, color extraction, and gradient thresholding
    warped_img = warp(image)
    masked_img = line_color_filter(warped_img)
    lines_img = lines_preprocessing(masked_img)
    
    # If this is the first frame, start searching for the lines
    if left_line.is_first_frame is True and right_line.is_first_frame is True:
        # Get the initial line fits from scratch
        success = get_line_fits(lines_img, left_line, right_line)
        left_line.detected = success
        right_line.detected = success

        left_line.is_first_frame = False
        right_line.is_first_frame = False
    # If we are not first frame
    else:
        # If we do not even have a previous fit or we are out of retries, start from scratch
        if (left_line.has_previous_fit() is False and right_line.has_previous_fit() is False) \
            or (left_line.retries == 0 and right_line.retries == 0):
            success = get_line_fits(lines_img, left_line, right_line)
            left_line.detected = success
            right_line.detected = success
            
            if success is True:
                left_line.reset_retries()
                right_line.reset_retries()
        # If we still have retries, continue from a previous fit
        elif left_line.retries > 0 and right_line.retries > 0:
            # If we do not have a previous successful fit, this is a retry
            if left_line.detected is False and right_line.detected is False:
                left_line.retries -= 1
                right_line.retries -= 1
            
            success = get_next_line_fits(lines_img, left_line, right_line)
            left_line.detected = success
            right_line.detected = success
            
            # If we were successful in this attempt to find a fit, reset the number of retries
            if success is True:
                left_line.reset_retries()
                right_line.reset_retries()
            
        # If we went over the retry limit and still didn't detect lines, start searching for lines from scratch again
        if left_line.retries == 0 and right_line.retries == 0 \
            and left_line.detected is False and right_line.detected is False:
            left_line.reset_retries()
            right_line.reset_retries()
            # Search for lines from scratch
            success = get_line_fits(lines_img, left_line, right_line)
            left_line.detected = success
            right_line.detected = success
    
    # Only draw lines if we have a fit (current or previous) to draw
    if left_line.has_previous_fit() is True and right_line.has_previous_fit() is True:
        result = draw_lines(image, lines_img, left_line, right_line)
    else:
        # We are not able to draw any lines because we do not have any fits
        result = image

    get_curvature(left_line, right_line)
    get_center_offset(image, left_line, right_line)
    
    result = write_measurements(result, left_line, right_line)

    return result

In [19]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

### Project Video

In [32]:
left_line = None
right_line = None

In [33]:
output = 'result.mp4'
clip1 = VideoFileClip("project_video.mp4")
result_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time result_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video result.mp4
[MoviePy] Writing video result.mp4


100%|██████████████████████████████████████████████████████▉| 1260/1261 [06:10<00:00,  3.26it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result.mp4 

Wall time: 6min 11s


In [34]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Challenge Video

In [38]:
left_line = None
right_line = None

In [39]:
output = 'result_challenge.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
result_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time result_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video result_challenge.mp4
[MoviePy] Writing video result_challenge.mp4


100%|█████████████████████████████████████████████████████████| 485/485 [02:24<00:00,  3.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_challenge.mp4 

Wall time: 2min 25s


In [40]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Harder Challenge Video

In [35]:
left_line = None
right_line = None

In [36]:
output = 'result_harder_challenge.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
result_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time result_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video result_harder_challenge.mp4
[MoviePy] Writing video result_harder_challenge.mp4


100%|██████████████████████████████████████████████████████▉| 1199/1200 [06:12<00:00,  3.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_harder_challenge.mp4 

Wall time: 6min 13s


In [37]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))